In [2]:
import pandas
import plotly
import urllib
from lxml import etree
import pandas as pd
import numpy as np

base_url = "http://www.rowclock.com/Results/Time?raceId="

def get_index(l, key):
    index = None
    try:
        index = l.index(key)
    except:
        pass
    return index
    


def extract_data(number):
    data = []
    headers = []
    positions = []
    race_name = ""
    try:
        f = urllib.request.urlopen(base_url + str(number))
        page = f.read()
        table = etree.HTML(page).find("body/div/table")
        race_name = etree.HTML(page).find(".//title").text.split(" - ")[0]
        rows = iter(table)
        headers = [str(col.text).strip("\r\n                ") for col in next(rows)]
        total_index = get_index(headers, "Total")
        raw_index = get_index(headers, "Raw Time")
        end_index = get_index(headers, "End")
        for row in rows:
            values = [col.text.strip("\r\n                ") for col in row]
            if (total_index and pd.Timedelta(values[total_index]).seconds == 0):
                values[total_index] = np.nan
            if (raw_index and pd.Timedelta(values[raw_index]).seconds == 0):
                values[raw_index] = np.nan
            if (end_index and pd.Timedelta(values[end_index]).seconds == 0):
                values[end_index] = np.nan
#             values.append(race_name)
            data.append(values)

    except:
        pass
    index=[int(pos[0]) for pos in data]
    frame = pd.DataFrame(data, columns=headers, index=index)
    return frame, race_name
extract_data

<function __main__.extract_data(number)>

In [3]:
frames = {}
for number in range(58):
    try:
        frame, name = extract_data(number)
        if name != "" and not(frame.empty):
            frames[name] = frame
            print(number, name), 
    except:
        pass
    
frames

9 Winter Head 2015
14 Winter Head to Head 2016
20 CCRC Spring H2H 2016
22 Wieertse 500
29 Christmas Head 2016
32 Winter League 2017
34 CCRC Winter Head to Head 2017 
35 Champs Junior Sculling Head 2017
36 Spring Head to Head 2017
37 Champs Eights Head 2017
39 Xpress Head 2017
40 Christmas Head 2017
43 Winter League 2018
45 Winter Head 2 Head 2018
49 Head of the Cam 2018
50 Spring H2H 2018
52 Champs Eights Head 2018
53 X-Press Head 2018
54 CCRC Christmas Head 2018
55 Winter League 2019
57 CCRC Winter H2H 2019


{'Winter Head 2015':      Pos  Num                     Name                        Club  \
 1      1  120                       M1             Queens' College   
 2      2    2                       M1          Downing College BC   
 3      3   81                       M1  Robinson College Boat Club   
 4      4   46                       M1     Jesus College Boat Club   
 5      5  117          M1 - Inteceptor               Lady Margaret   
 6      6  119                       M1         Pembroke College BC   
 7      7    1                       M1           City of Cambridge   
 8      8  118                       M1             First and Third   
 9      9  121                       M1        Churchill College BC   
 10    10   49                       M1         Wolfson (Cambridge)   
 11    11   79                       M1                    Emmanuel   
 12    12  122                Darwin M1           Darwin College BC   
 13    13   48                       M1           Girton 

In [5]:
clare_names = ["Clare", "Clare BC", "Clare Boat Club"]
clare_m1_finish = None
for name, frame in frames.items():
    clare = frame.Club.isin(clare_names)
    m1 = frame["Name"] == "M1"
    if frame[clare].size > 0:
        frame["Race"] = name
        new_data = (frame[clare])[['Pos','Name', 'Race', 'Total', 'Type', 'Raw Time']]
        if clare_m1_finish is None:
            clare_m1_finish = new_data
        else:
            clare_m1_finish = clare_m1_finish.append(new_data, )
            print(new_data)
clare_m1_finish

   Pos Name                  Race      Total Type   Raw Time
1    1   M1  CCRC Spring H2H 2016  0:12:47.1   8+  0:12:47.1
9    9   M2  CCRC Spring H2H 2016  0:13:50.5   8+  0:13:50.5
22  22   W1  CCRC Spring H2H 2016  0:15:04.6   8+  0:15:04.6
23  23   M3  CCRC Spring H2H 2016  0:15:05.3   8+  0:15:05.3
64  64   W2  CCRC Spring H2H 2016  0:16:50.7   8+  0:16:50.7
83  83   W3  CCRC Spring H2H 2016  0:17:40.2   8+  0:17:40.2
     Pos               Name                 Race      Total  Type   Raw Time
21    21               2+2x  Christmas Head 2016  0:06:58.8  2+2x  0:06:58.8
44    44  French Connection  Christmas Head 2016  0:07:27.9    2-  0:07:27.9
171  171               16++  Christmas Head 2016        NaN  16++        NaN
     Pos      Name                            Race      Total Type   Raw Time
9      9  Clare M1  CCRC Winter Head to Head 2017   0:13:45.1   8+  0:13:45.1
21    21  Clare M2  CCRC Winter Head to Head 2017   0:14:42.2   8+  0:14:42.2
51    51  Clare M3  CCRC Winter

,Pos,Name,Race,Total,Type,Raw Time
46,46,M2,Winter Head 2015,0:15:45.0,8+,0:15:45.0
88,88,W1,Winter Head 2015,0:16:56.0,8+,0:16:56.0
118,118,W2,Winter Head 2015,0:18:00.0,8+,0:18:00.0
152,152,M3,Winter Head 2015,NaN,8+,NaN
155,155,W3,Winter Head 2015,NaN,8+,NaN
1,1,M1,CCRC Spring H2H 2016,0:12:47.1,8+,0:12:47.1
9,9,M2,CCRC Spring H2H 2016,0:13:50.5,8+,0:13:50.5
22,22,W1,CCRC Spring H2H 2016,0:15:04.6,8+,0:15:04.6
23,23,M3,CCRC Spring H2H 2016,0:15:05.3,8+,0:15:05.3
64,64,W2,CCRC Spring H2H 2016,0:16:50.7,8+,0:16:50.7


In [11]:
clare_names = ["M1", "Clare M1"]
clare_m1_finish = clare_m1_finish[clare_m1_finish.Name.isin(clare_names)]
clare_m1_finish

AttributeError: 'Series' object has no attribute 'Name'